In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 30. GraphFrame 
* edge와 node (또는 vertex) 로 구성된 그래프 데이터 구조를 표현하기 위한 객체로 
* edge에 대한 속성을 담은 데이터 프레임과 node 의 속성을 담은 데이터 프레임 2개로 구성됨 
* fraud 에서 계좌간의 거래 속성을 표현하거나, pagerank 에서 페이지 간의 관계를 파악하거나, 친구관계를 분석할 때 그래프로 표현하고 계산하는 것이 적합할 수 있음
* Spark의 GraphFrame 은 GraphX 를 확장하여 DataFrame API 를 제공하고 여러 언어를 지원함

### 30.1 GraphFrame 만들기 
#### 30.1.1 Load Data
* vertex/node (기차역) 과 edge (출발 기차역 -> 도착 기차역의 관계) 의 속성을 담은 두개의 데이터 프레임을 로드함 

In [2]:
bikeStations = spark.read.option("header","true").csv("/Users/tuhm/Documents/Git_repo/Spark-The-Definitive-Guide/data/bike-data/201508_station_data.csv")
tripData = spark.read.option("header","true").csv("/Users/tuhm/Documents/Git_repo/Spark-The-Definitive-Guide/data/bike-data/201508_trip_data.csv")


#### 30.1.2 Column Rename
GraphFrame 에서 제시하는 컬럼 명명 규칙을 따르기 위해 rename
* vertex/node 는 식별자를 id 로 정의하고 edge는 src (시작 node id), 와 dst (도착 node id) 를 가져야 한다

In [3]:
stationVertices = bikeStations.withColumnRenamed("name", "id").distinct()
tripEdges = tripData.withColumnRenamed("Start Station", "src").withColumnRenamed("End Station", "dst")


In [4]:
stationVertices.show(5)

+----------+--------------------+---------+-----------+---------+-------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|
+----------+--------------------+---------+-----------+---------+-------------+------------+
|        51|Embarcadero at Fo...|37.791464|-122.391034|       19|San Francisco|   8/20/2013|
|        58|San Francisco Cit...| 37.77865|-122.418235|       19|San Francisco|   8/21/2013|
|        60|Embarcadero at Sa...| 37.80477|-122.403234|       15|San Francisco|   8/21/2013|
|        65|     Townsend at 7th|37.771058|-122.402717|       15|San Francisco|   8/22/2013|
|        63|       Howard at 2nd|37.786978|-122.398108|       19|San Francisco|   8/22/2013|
+----------+--------------------+---------+-----------+---------+-------------+------------+
only showing top 5 rows



In [5]:
tripEdges.show(5)

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

#### 30.1.3. Create GraphFrame
pyspark 실행시 아래의 커맨드로 실행시켜야 라이브러리를 로드할 수 있음 (!주의! 스파크 버전 (spark3.1) 과 scala 버전 (s_2.12) 를 확인하여 적절한 것으로 로드)
> $ ./bin/pyspark --packages graphframes:graphframes:0.8.0-spark3.1-s_2.12

In [6]:
from graphframes import GraphFrame
stationGraph = GraphFrame(stationVertices, tripEdges)
stationGraph.cache()


GraphFrame(v:[id: string, station_id: string ... 5 more fields], e:[src: string, dst: string ... 9 more fields])

GraphFrame 은 두개의 DataFrame 을 가지고 있다고 생각하면 됨 
* .vertices 로 접근할 수 있는 vertex/node (정점) 에 대한 dataframe 
* .edges 로 접근할 수 있는 edge 에 대한 dataframe 

In [7]:
stationGraph.vertices.show(5)

+----------+--------------------+---------+-----------+---------+-------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|
+----------+--------------------+---------+-----------+---------+-------------+------------+
|        51|Embarcadero at Fo...|37.791464|-122.391034|       19|San Francisco|   8/20/2013|
|        58|San Francisco Cit...| 37.77865|-122.418235|       19|San Francisco|   8/21/2013|
|        60|Embarcadero at Sa...| 37.80477|-122.403234|       15|San Francisco|   8/21/2013|
|        65|     Townsend at 7th|37.771058|-122.402717|       15|San Francisco|   8/22/2013|
|        63|       Howard at 2nd|37.786978|-122.398108|       19|San Francisco|   8/22/2013|
+----------+--------------------+---------+-----------+---------+-------------+------------+
only showing top 5 rows



In [8]:
stationGraph.edges.show(5)

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

### 30.2 GraphFrame 쿼리하기 
두개의 데이터 프레임을 품고 있기 때문에 각각을 access해서 DataFrame 에 수행할 수 있는 모든 쿼리를 수행할 수 있음 
* groupby count on stationGraph.edges
* filter(where) and groupby count on stationGraph.edges

In [35]:
from pyspark.sql.functions import desc
stationGraph.edges.groupBy("src", "dst").count().orderBy(desc("count")).show(10, False)

+---------------------------------------------+----------------------------------------+-----+
|src                                          |dst                                     |count|
+---------------------------------------------+----------------------------------------+-----+
|San Francisco Caltrain 2 (330 Townsend)      |Townsend at 7th                         |3748 |
|Harry Bridges Plaza (Ferry Building)         |Embarcadero at Sansome                  |3145 |
|2nd at Townsend                              |Harry Bridges Plaza (Ferry Building)    |2973 |
|Townsend at 7th                              |San Francisco Caltrain 2 (330 Townsend) |2734 |
|Harry Bridges Plaza (Ferry Building)         |2nd at Townsend                         |2640 |
|Embarcadero at Folsom                        |San Francisco Caltrain (Townsend at 4th)|2439 |
|Steuart at Market                            |2nd at Townsend                         |2356 |
|Embarcadero at Sansome                       |Ste

In [42]:
stationGraph.edges.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'").groupBy("src", "dst").count().orderBy(desc("count")).show(10, False)


+---------------------------------------------+---------------------------------------------+-----+
|src                                          |dst                                          |count|
+---------------------------------------------+---------------------------------------------+-----+
|San Francisco Caltrain 2 (330 Townsend)      |Townsend at 7th                              |3748 |
|Townsend at 7th                              |San Francisco Caltrain 2 (330 Townsend)      |2734 |
|Townsend at 7th                              |San Francisco Caltrain (Townsend at 4th)     |2192 |
|Townsend at 7th                              |Civic Center BART (7th at Market)            |1844 |
|Civic Center BART (7th at Market)            |Townsend at 7th                              |1765 |
|San Francisco Caltrain (Townsend at 4th)     |Townsend at 7th                              |1198 |
|Temporary Transbay Terminal (Howard at Beale)|Townsend at 7th                              |834  |


#### 30.2.1 서브 그래프
* edge 또는 node 에 해당하는 DataFrame 을 where 조건으로 필터링 해서 새로운 GraphFrame 을 만듬 
* 부분집합으로 구성하기 때문에 서브 그래프라고 부름

In [11]:
townAnd7thEdges = stationGraph.edges.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")
subgraph = GraphFrame(stationGraph.vertices, townAnd7thEdges)


### 30.3 모티브 찾기 

* 모티브란 구조적 패턴을 그래프로 표현하는 것을 말함 (예: a->b->c->a 로 이어지는 삼각형 패턴을 형성하는 path 를 찾기)
* GraphFrame 을 DataFrame 취급하여, Neo4J의 Cypher 언어를 활용한 (특이한) 쿼리가 가능
* (a)-[ab]->(b) : node (a) 에서 edge[ab] 를 통해 node (b)로 감 
    ** 결과에 반환할 이름으로 a, b 를 지정하는 것이지 (a)-[]->(b) 도 가능

In [12]:
#motifs = stationGraph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)") : local 에서 너무 오래 걸림
motifs = stationGraph.find("(a)-[ab]->(b); (b)-[ba]->(a)")

In [31]:
print(motifs.count())

254170794


In [28]:
motifs.show(5)

+--------------------+--------------------+--------------------+--------------------+
|                   a|                  ab|                   b|                  ba|
+--------------------+--------------------+--------------------+--------------------+
|{70, San Francisc...|{912347, 550, 8/3...|{62, 2nd at Folso...|{913046, 309, 8/3...|
|{70, San Francisc...|{912347, 550, 8/3...|{62, 2nd at Folso...|{912792, 509, 8/3...|
|{70, San Francisc...|{912347, 550, 8/3...|{62, 2nd at Folso...|{912751, 360, 8/3...|
|{70, San Francisc...|{912347, 550, 8/3...|{62, 2nd at Folso...|{911828, 525, 8/3...|
|{70, San Francisc...|{912347, 550, 8/3...|{62, 2nd at Folso...|{911390, 711, 8/3...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [47]:
motifs.groupby("a").count().orderBy(desc("count")).show(3, False)

+---------------------------------------------------------------------------------------------------+--------+
|a                                                                                                  |count   |
+---------------------------------------------------------------------------------------------------+--------+
|{70, San Francisco Caltrain (Townsend at 4th), 37.776617, -122.39526, 19, San Francisco, 8/23/2013}|34942229|
|{69, San Francisco Caltrain 2 (330 Townsend), 37.7766, -122.39547, 23, San Francisco, 8/23/2013}   |27495219|
|{50, Harry Bridges Plaza (Ferry Building), 37.795392, -122.394203, 23, San Francisco, 8/20/2013}   |22037787|
+---------------------------------------------------------------------------------------------------+--------+
only showing top 3 rows



#### 이 자체로 데이터 프레임이기 때문에 여러가지 쿼리가 가능함 (아래는 a에서 b, c 그리고 a로 이동한 가장 짧은 경로를 찾게 됨)

또한 거의 반드시 모티브에서 반환한 결과를 필터링 해야 함 (동일한 정점 ID 가 여러개 있는 경우 등) 
* 동일한 자전거로 이동한 경로만 추출(``.where("ca.`Bike #` = bc.`Bike #`").where("ab.`Bike #` = bc.`Bike #`")``)
* a와 b, c 가 서로 다른 지점인지 (`.where("a.id != b.id").where("b.id != c.id")`)
* 이동 순서가 a 에서 b, b에서 c 로 가는지(`.where("abStart < bcStart").where("bcStart < caStart")`)

In [53]:
from pyspark.sql.functions import expr
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

motifs.selectExpr("*",
    "to_timestamp(ab.`Start Date`, 'MM/dd/yyyy HH:mm') as abStart",
    "to_timestamp(ba.`Start Date`, 'MM/dd/yyyy HH:mm') as baStart").where(
    "ab.`Bike #` = ba.`Bike #`").where(
    "a.id != b.id").where(
    "abStart < baStart").orderBy(
    expr("cast(baStart as long) - cast(abStart as long)")).limit(1).show(1)


+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|                   a|                  ab|                   b|                  ba|            abStart|            baStart|
+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|{65, Townsend at ...|{848055, 78, 7/15...|{64, 2nd at South...|{848088, 782, 7/1...|2015-07-15 16:57:00|2015-07-15 16:59:00|
+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+



In [54]:
motifs.selectExpr("*",
    "to_timestamp(ab.`Start Date`, 'MM/dd/yyyy HH:mm') as abStart",
    "to_timestamp(ba.`Start Date`, 'MM/dd/yyyy HH:mm') as baStart").where(
    "ab.`Bike #` = ba.`Bike #`").where(
    "a.id != b.id").where(
    "abStart < baStart").orderBy( # 가장 오랜시간이 걸린 path 를 찾기
    desc(expr("cast(baStart as long) - cast(abStart as long)"))).limit(1).show(1)


+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|                   a|                  ab|                   b|                  ba|            abStart|            baStart|
+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|{60, Embarcadero ...|{433064, 1023, 9/...|{74, Steuart at M...|{911831, 338, 8/3...|2014-09-01 11:34:00|2015-08-31 08:01:00|
+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+



### 30.4 그래프에 적용할 수 있는 알고리즘 

#### 30.4.1 페이지 랭크
* 웹페이지의 순위를 정하는 알고리즘으로,
* 다른 웹페이지에 많이 링크되어 있을 경우 더 중요하다고 판단함 
* 자전거 path 데이터에 적용해보면 자전거가 많이 거쳐가는 지점이 더 중요하다고 볼 수 있으며, 친구 관계에서는 메세지를 많이 주고받는 유저가 상위에 랭크될 수 있다

페이지 랭크 메쏘드의 결과로는 GraphFrame 이 반환되어 여기에 `.vertices` 나 `.edges` 로 access할 수 있고, 새롭게 pagerank 라는 컬럼이 생성됨 

In [55]:
from pyspark.sql.functions import desc
ranks = stationGraph.pageRank(resetProbability=0.15, maxIter=10)

ranks.vertices.orderBy(desc("pagerank")).select("id", "pagerank").show(10)


+--------------------+------------------+
|                  id|          pagerank|
+--------------------+------------------+
|San Jose Diridon ...| 4.051504835990019|
|San Francisco Cal...|3.3511832964286965|
|Mountain View Cal...|2.5143907710155435|
|Redwood City Calt...| 2.326308771371171|
|San Francisco Cal...| 2.231144291369883|
|Harry Bridges Pla...|1.8251120118882473|
|     2nd at Townsend|1.5821217785038688|
|Santa Clara at Al...|1.5730074084907584|
|     Townsend at 7th|1.5684565805340545|
|Embarcadero at Sa...|1.5414242087748589|
+--------------------+------------------+
only showing top 10 rows



#### 30.4.2 In-Degree와 Out-Degree
* In-Degree: 주어진 지점 (node) 를 도착점(dst) 으로 갖는 edge 갯수 
* Out-Degree: 주어진 지점 (node) 를 출발점(src) 로 갖는 edge 갯수
* Social network 에서는 특정 유저의 follower 수는 In-Degree 가 되고, following 수는 Out-Degree가 됨 

In [56]:
inDeg = stationGraph.inDegrees
inDeg.orderBy(desc("inDegree")).show(5, False)


+----------------------------------------+--------+
|id                                      |inDegree|
+----------------------------------------+--------+
|San Francisco Caltrain (Townsend at 4th)|34810   |
|San Francisco Caltrain 2 (330 Townsend) |22523   |
|Harry Bridges Plaza (Ferry Building)    |17810   |
|2nd at Townsend                         |15463   |
|Townsend at 7th                         |15422   |
+----------------------------------------+--------+
only showing top 5 rows



In [57]:
outDeg = stationGraph.outDegrees
outDeg.orderBy(desc("outDegree")).show(5, False)


+---------------------------------------------+---------+
|id                                           |outDegree|
+---------------------------------------------+---------+
|San Francisco Caltrain (Townsend at 4th)     |26304    |
|San Francisco Caltrain 2 (330 Townsend)      |21758    |
|Harry Bridges Plaza (Ferry Building)         |17255    |
|Temporary Transbay Terminal (Howard at Beale)|14436    |
|Embarcadero at Sansome                       |14158    |
+---------------------------------------------+---------+
only showing top 5 rows



두 값의 비율로, following 수 대비 follower 수가 많은 비율을 구해볼 수 있음 (인기 많은 유저 or influencer)

In [58]:
degreeRatio = inDeg.join(outDeg, "id")\
  .selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio")
degreeRatio.orderBy(desc("degreeRatio")).show(10, False)

+----------------------------------------+------------------+
|id                                      |degreeRatio       |
+----------------------------------------+------------------+
|Redwood City Medical Center             |1.5333333333333334|
|San Mateo County Center                 |1.4724409448818898|
|SJSU 4th at San Carlos                  |1.3621052631578947|
|San Francisco Caltrain (Townsend at 4th)|1.3233728710462287|
|Washington at Kearny                    |1.3086466165413533|
|Paseo de San Antonio                    |1.2535046728971964|
|California Ave Caltrain Station         |1.24              |
|Franklin at Maple                       |1.2345679012345678|
|Embarcadero at Vallejo                  |1.2201707365495336|
|Market at Sansome                       |1.2173913043478262|
+----------------------------------------+------------------+
only showing top 10 rows



#### 30.4.3 너비 우선 탐색 (Breath-First Search)

* 두개의 노드 (fromExpr, toExpr) 를 연결하는 edge 의 조합을 탐색하는 방법으로, BFS를 통해 가장 짧은 (edge의 수가 적은) path 를 찾을 수 있음
    - maxPathLength = 최대로 갖을 수 있는 에지의 수 
    - edgeFilter: 조건에 맞지 않는 에지를 필터링
* BFS를 통해 찾은 가장 짧은 path 가 edge 3개의 조합이라면, 아래의 형태로 DataFrame 이 return 됨 (중간에 거쳐가는 지점이 v1과 v2) 
> from | e0 | v1 | e1 | v2 | e2 | to
* maxPathLength 에 도달할 때까지 from, to 조건을 충족하는 path를 찾지 못하면 아무것도 return 되지 않고, length 가 동일한 path 가 여럿 발견되면 모두 return 됨  
* ref: https://graphframes.github.io/graphframes/docs/_site/api/scala/org/graphframes/lib/BFS.html

##### Breath-First Search 란
* Depth-First 에 대비되는 개념으로 시작점 (a) 라고 할 때, level=1, 시작점 (a) 에 연결된 노드 (b1, b2, b3...) (level=2)를 전부 탐색하고 나서 그다음 단계 (level=3)로 넘어감 
* Depth-First Search 에서는 시작점 (a) 에서 연결된 노드 하나 (b1)를 찾고 난 뒤, (b1) 와 연결된 노드, (...) 로 더이상 연결된 노드가 없을 때까지 다 탐색하고 난뒤, (a) 와 연결된 다른 노드 (b2) 로 넘어감


In [63]:
stationGraph.bfs(fromExpr="id = 'Townsend at 7th'",
  toExpr="id = 'Embarcadero at Vallejo'", maxPathLength=3).show(10)


+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|{65, Townsend at ...|{903824, 985, 8/2...|{48, Embarcadero ...|
|{65, Townsend at ...|{901592, 1548, 8/...|{48, Embarcadero ...|
|{65, Townsend at ...|{897999, 1083, 8/...|{48, Embarcadero ...|
|{65, Townsend at ...|{893393, 1940, 8/...|{48, Embarcadero ...|
|{65, Townsend at ...|{893382, 1992, 8/...|{48, Embarcadero ...|
|{65, Townsend at ...|{893380, 2053, 8/...|{48, Embarcadero ...|
|{65, Townsend at ...|{884461, 1130, 8/...|{48, Embarcadero ...|
|{65, Townsend at ...|{876822, 760, 8/5...|{48, Embarcadero ...|
|{65, Townsend at ...|{859662, 4572, 7/...|{48, Embarcadero ...|
|{65, Townsend at ...|{855218, 1105, 7/...|{48, Embarcadero ...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



#### 30.4.4 Connected Components (연결 요소) 
* 방향성이 없는 그래프, src 와 dst의 개념이 아니라 node와 이를 연결하는 (방향성 없는) edge의 개념에서만 존재 (예제와는 안 맞음)
     * Connected Components 는 node 의 집합으로, 해당 집합 안에 있는 모든 node는 서로와 연결이 되어야 함 
     * 두개의 서로 다른 C.C 는 교집합이 없어 서로 연결되어 있지 않고, 하나의 그래프는 여러개의 C.C 로 합집합을 이룸
     * ref: https://www.baeldung.com/cs/graph-connected-components
     
* 아래의 그림에서는 3개의 connected component 만 발견된다
    * 가장 좌측에 (V1, V2, V3, E3, E4, E5) 는 서로 연결되어 있으나, 또 다른 노드들과 더 큰 연결을 이루기 때문에 C.C 가 아님
* 그래프 외에, Image Processing 에도 Connected Components라는 개념이 활용됨

<img src="img/connected_components.png" alt="cc" width="400" align="left"/>


* C.C 는 DFS 를 활용하고, 계산량이 가장 많아, 충돌이 발생하면 작업을 마쳤던 곳에서 다시 시작할 수 있도록 checkpoint 실행함 

In [64]:
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")

# 계산량이 많으므로 샘플링을 적극 이용한다
minGraph = GraphFrame(stationVertices, tripEdges.sample(False, 0.1))
cc = minGraph.connectedComponents()


* connectedComponents() 의 결과로 도출된 DataFrame 에는 node (station) 의 정보와 함께 component 라는 새로운 컬럼이 생성됨 
    * 서로 연결된 vertex/node 에는 같은 id 가 부여됨 

In [74]:
cc.groupby("component").count().show()


+------------+-----+
|   component|count|
+------------+-----+
|128849018880|   16|
|  8589934592|   19|
|           0|   33|
| 17179869184|    1|
|317827579904|    1|
+------------+-----+



In [79]:
cc.where("component == 0 ").show(5)

+----------+--------------------+---------+-----------+---------+-------------+------------+---------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|component|
+----------+--------------------+---------+-----------+---------+-------------+------------+---------+
|        51|Embarcadero at Fo...|37.791464|-122.391034|       19|San Francisco|   8/20/2013|        0|
|        58|San Francisco Cit...| 37.77865|-122.418235|       19|San Francisco|   8/21/2013|        0|
|        60|Embarcadero at Sa...| 37.80477|-122.403234|       15|San Francisco|   8/21/2013|        0|
|        65|     Townsend at 7th|37.771058|-122.402717|       15|San Francisco|   8/22/2013|        0|
|        63|       Howard at 2nd|37.786978|-122.398108|       19|San Francisco|   8/22/2013|        0|
+----------+--------------------+---------+-----------+---------+-------------+------------+---------+
only showing top 5 rows



#### 30.4.5 Strongly Connected Components (강한 연결 요소)
* Directed Graph 에서 나오는 개념으로 강하게 연결되었다 함은, 두개의 node x와 y에 대하여 x-> y와 y-> x의 경로가 모두 존재하는 경우를 말함 (되돌아갈 수 있는 경우)
* Strongly Connected Components는 이렇게 서로 Strongly Connected된 Vertex/Node 들의 집합(또는 Subgraph)를 말한다.

In [69]:
scc = minGraph.stronglyConnectedComponents(maxIter=3)
scc.show(5)

+----------+--------------------+---------+-----------+---------+-------------+------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|   component|
+----------+--------------------+---------+-----------+---------+-------------+------------+------------+
|        11|         MLK Library|37.335885| -121.88566|       19|     San Jose|    8/6/2013|128849018880|
|        80|Santa Clara Count...|37.352601|-121.905733|       15|     San Jose|  12/31/2013|128849018880|
|        64|   2nd at South Park|37.782259|-122.392738|       15|San Francisco|   8/22/2013|           0|
|        36|California Ave Ca...|37.429082|-122.142805|       15|    Palo Alto|   8/14/2013|  8589934592|
|        62|       2nd at Folsom|37.785299|-122.396236|       19|San Francisco|   8/22/2013|           0|
+----------+--------------------+---------+-----------+---------+-------------+------------+------------+
only showing top 5 rows



#### 30.4.6 번외

그 외 책에는 나오지 않았지만, triangleCounts() 나 shortestPaths 라는 알고리즘도 있음 